En primer lugar cargamos la página web para proceder a su extracción.

In [1]:
import requests

URL = "http://quotes.toscrape.com/"
page = requests.get(URL)

# print(page.text)

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content, "html.parser")

# soup

In [3]:
results = soup.find(class_="container")

# print(results.prettify())

Extraemos en primer lugar cada una de las citas para cada una de las páginas.

In [28]:
quote_elements = results.find_all("span", class_="text")

current_url = URL

texts = []

while True:
    page = requests.get(current_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="container")

    for quote_element in quote_elements:
        text = quote_element.get_text(strip=True)
        texts.append(text)
        
    next_button = soup.find("li", class_="next")
    
    if next_button:
        current_url = URL + next_button.find("a")["href"]
    else:
        break
    
len(texts)

100

Procedemos de la misma manera con los autores.

In [27]:
author_elements = results.find_all("small", class_="author")

URL = "http://quotes.toscrape.com/"
current_URL = URL
authors = []

while True:
    page = requests.get(current_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="container")
    
    for author_element in author_elements:
        author = author_element.get_text(strip=True)
        authors.append(author)
    
    next_button = soup.find("li", class_="next")
        
    if next_button:
        current_URL = URL + next_button.find("a")["href"]
    else:
        break

len(authors)

100

Y finalmente con las etiquetas.

In [30]:
total_elements = results.find_all("div", class_="quote")

URL = "http://quotes.toscrape.com/"
current_URL = URL
tags = []

while True:
    page = requests.get(current_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(class_="container")
    
    for tag_element in total_elements:
        tag = tag_element.find('div', class_='tags').meta['content']
        #tag = tag_element.get_text(strip=True)
        tags.append(tag)
    
    next_button = soup.find("li", class_="next")
        
    if next_button:
        current_URL = URL + next_button.find("a")["href"]
    else:
        break

len(tags)

100

Finalmente procedemos a juntar en un mismo dataset toda la información extraída individualmente.

In [37]:
import pandas as pd

df_quotes = pd.DataFrame({'Author': authors, 'Quote': texts, 'Tags': tags})
df_quotes

,Author,Quote,Tags
0,Albert Einstein,“The world as we have created it is a process ...,"change,deep-thoughts,thinking,world"
1,J.K. Rowling,"“It is our choices, Harry, that show what we t...","abilities,choices"
2,Albert Einstein,“There are only two ways to live your life. On...,"inspirational,life,live,miracle,miracles"
3,Jane Austen,"“The person, be it gentleman or lady, who has ...","aliteracy,books,classic,humor"
4,Marilyn Monroe,"“Imperfection is beauty, madness is genius and...","be-yourself,inspirational"
...,...,...,...
95,Harper Lee,“You never really understand a person until yo...,better-life-empathy
96,Madeleine L'Engle,“You have to write the book that wants to be w...,"books,children,difficult,grown-ups,write,write..."
97,Mark Twain,“Never tell the truth to people who are not wo...,truth
98,Dr. Seuss,"“A person's a person, no matter how small.”",inspirational


Ahora procedemos a realizar lo mismo, pero con una tabla de los microestados del mundo en un artículo de la wikipedia.

In [38]:
URL_wiki = "https://en.wikipedia.org/wiki/List_of_micronations"

page_wiki = requests.get(URL_wiki)

In [39]:
soup_wiki = BeautifulSoup(page_wiki.content, "html.parser")

# soup_wiki

En este caso, la extracción de la tabla con Beautiful Soup y podemos localizar la tabla con una sencilla instrucción, para después formar un dataframe y así poderla visualizar.

In [40]:
results_wiki = soup_wiki.find_all('table', {'class': 'wikitable'})

# results_wiki

In [45]:
df = pd.read_html(str(results_wiki))

df = pd.DataFrame(df[0])
df

,Flag,Foundation,Name,Description
0,Antarctica,Antarctica,Antarctica,Antarctica
1,NaN,2008,Grand Duchy of Flandrensis,"Founded as a short-term hobby,[2] Flandrensis ..."
2,NaN,2001,Grand Duchy of Westarctica,"Founded by Travis McHenry, Westarctica claims ..."
3,Antigua and Barbuda,Antigua and Barbuda,Antigua and Barbuda,Antigua and Barbuda
4,NaN,1865,Kingdom of Redonda,"Redonda is a small, uninhabitable island south..."
...,...,...,...,...
76,Link to file,2000,Global Country of World Peace,"Maharishi Mahesh Yogi, the founder of Transcen..."
77,NaN,1990,Dominion of Melchizedek,This source of financial fraud[60] claims seve...
78,NaN,1998,Nova Roma,Nova Roma is an organization that seeks to rec...
79,NaN,1973,Nutopia,"John Lennon and Yoko Ono founded this ""concept..."
